In [95]:
import numpy as np
import numpy.ma as ma
import pymc3 as pm
import theano.tensor as tt
import os
import csv
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import multiprocessing as mp

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

SEED = 350107321 # from random.org
np.random.seed(SEED)

print(plt.style.available)
plt.style.use('seaborn-white')

RuntimeError: context has already been set

In [55]:
def cubicsolver(coef):
    
    a = coef[0]
    b = coef[1]
    c = coef[2]
    d = coef[3]

    f = findF(a, b, c)                          # Helper Temporary Variable
    g = findG(a, b, c, d)                       # Helper Temporary Variable
    h = findH(g, f)                             # Helper Temporary Variable

    i = np.sqrt(((g ** 2.0) / 4.0) - h)   # Helper Temporary Variable
    j = i ** (1 / 3.0)                      # Helper Temporary Variable
    k = np.arccos(-(g / (2 * i)))           # Helper Temporary Variable
    L = j * -1                              # Helper Temporary Variable
    M = np.cos(k / 3.0)                   # Helper Temporary Variable
    N = np.sqrt(3) * np.sin(k / 3.0)    # Helper Temporary Variable
    P = (b / (3.0 * a)) * -1                # Helper Temporary Variable

    x1 = 2 * j * np.cos(k / 3.0) - (b / (3.0 * a))
    x2 = L * (M + N) + P
    x3 = L * (M - N) + P

    return np.array([x1, x2, x3])           # Returning Real Roots as numpy array.
# Helper function to return float value of f.
def findF(a, b, c):
    return ((3.0 * c / a) - ((b ** 2.0) / (a ** 2.0))) / 3.0


# Helper function to return float value of g.
def findG(a, b, c, d):
    return (((2.0 * (b ** 3.0)) / (a ** 3.0)) - ((9.0 * b * c) / (a **2.0)) + (27.0 * d / a)) /27.0


# Helper function to return float value of h.
def findH(g, f):
    return ((g ** 2.0) / 4.0 + (f ** 3.0) / 27.0)

In [6]:
currentdir = os.getcwd()
datadir = os.path.join(currentdir,'OxytocinRSData_new')
print(datadir)

/Users/hstrey/Documents/programming/fMRI-analysis/OxytocinRSData_new


In [10]:
# get data files names
datafilenames = os.listdir(datadir)
datafilenames.sort()

['.DS_Store',
 'Subject001.npy',
 'Subject002.npy',
 'Subject003.npy',
 'Subject004.npy',
 'Subject005.npy']

In [12]:
time_series = np.load(os.path.join(datadir,datafilenames[1]),allow_pickle=True)
print(time_series.shape)

(748, 32)


In [13]:
# 748 = 2x2x11x17 so splitting it up into chuncks of 44 is a good idea
WINDOW_LENGTH =44
num = int((time_series.shape[0] - 1 - \
               (time_series.shape[0] - 1) % WINDOW_LENGTH) /
              WINDOW_LENGTH)
indicies = np.linspace(WINDOW_LENGTH, num*WINDOW_LENGTH, num).astype(int)

windows = np.split(time_series, indicies, axis=0)

In [18]:
windows = np.array(windows)

In [19]:
windows.shape

(17, 44, 32)

In [20]:
one = windows[0,:,:]
two = windows[1,:,:]

In [94]:
np.tri(3,3,k=-1)

array([[0., 0., 0.],
       [1., 0., 0.],
       [1., 1., 0.]])

In [117]:
# we now need to create a 32x32 matrix of correlations
b = np.repeat(one[:, :, np.newaxis], 32, axis=2)*np.tri(32,32,-1)
c = np.repeat(one[:, :, np.newaxis], 32, axis=2).swapaxes(1,2)*np.tri(32,32,-1)
mask = np.repeat(np.tri(32,32,-1)[np.newaxis,:, :],44,axis=0)
print(b.shape,c.shape,mask.shape)

(44, 32, 32) (44, 32, 32) (44, 32, 32)


In [118]:
bm = ma.masked_array(b, mask=mask)
cm = ma.masked_array(c, mask=mask)
type(bm)

numpy.ma.core.MaskedArray

In [128]:
y1 = b + c
y2 = b - c

In [129]:
def calcFundamentalStats(y):
    ass = np.sum(y[1:-1]**2,axis=0)
    aep = y[0]**2 + y[-1]**2
    ac = np.sum(y[0:-1]*y[1:],axis=0)
    return aep,ass,ac

In [73]:
def calcBfromDataN(aep,ass,ac,N):
    coef = np.array([(N-1)*ass,
       (2.0-N)*ac,
       -aep-(N+1)*ass,
       N*ac])
    b = cubicsolver(coef)
    return b[2,:,:]

In [130]:
aep1,ass1,ac1 = calcFundamentalStats(y1)
aep2,ass2,ac2 = calcFundamentalStats(y2)
print(ass1.shape,aep1.shape,ac1.shape)

(32, 32) (32, 32) (32, 32)


In [131]:
B1 = calcBfromDataN(aep1,ass1,ac1,44)
B2 = calcBfromDataN(aep2,ass2,ac2,44)

In [104]:
B1.shape

(32, 32)

In [70]:
def calcAfromB(B,aep,ass,ac):
    Q=aep1/(1-B**2)
    Q=Q+ass*(1+B**2)/(1-B**2)
    Q=Q-ac*2*B/(1-B**2)
    A = Q/N
    P2A = -N/A**2/2
    Btmp = B**2*(1+2*N)
    tmp = (1+Btmp)*aep + (2*Btmp + N + 1 -B**4*(N-1))*ass - 2*B*(1+B**2+2*N)*ac
    P2B = -tmp/((1-B**2)**2*(aep1 + (1+B**2)*ass - 2*B*ac))
    PAB = (N-1)*B/A/(1-B**2)
    dA = np.sqrt(-P2B/(P2A*P2B-PAB**2))
    dB = np.sqrt(-P2A/(P2A*P2B-PAB**2))
    return A,dA,dB

In [132]:
A1,dA1,dB1 = calcAfromB(B1,aep1,ass1,ac1)
A2,dA2,dB2 = calcAfromB(B2,aep2,ass2,ac2)

In [133]:
Adiff = A1-A2

In [134]:
C = np.where(Adiff>0,Adiff/A2,Adiff/A1)

In [135]:
dC = np.where(Adiff>0,np.sqrt(dA1**2/A1**2 + A1**2*dA2**2/A2**4),np.sqrt(dA2**2/A1**2 + A2**2*dA1**2/A1**4))

In [142]:
C

array([[        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [-0.01008025,         nan,         nan, ...,         nan,
                nan,         nan],
       [-0.07917531,  2.48333215,         nan, ...,         nan,
                nan,         nan],
       ...,
       [ 0.99915326,  0.11143428,  0.2220165 , ...,         nan,
                nan,         nan],
       [ 0.30345814, -0.29584338,  0.29894772, ...,  0.37308003,
                nan,         nan],
       [-0.20023392, -0.1609543 ,  0.47145325, ...,  0.14045303,
         1.01829989,         nan]])

In [141]:
dC

array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [0.4287121 ,        nan,        nan, ...,        nan,        nan,
               nan],
       [0.40600708, 1.33738398,        nan, ...,        nan,        nan,
               nan],
       ...,
       [0.75318171, 0.54084688, 0.77343743, ...,        nan,        nan,
               nan],
       [0.38156371, 0.47241831, 0.37336537, ..., 0.53120174,        nan,
               nan],
       [0.41311191, 0.56943747, 1.41862497, ..., 0.63444925, 0.51344271,
               nan]])